# Implementing Gradient Descent and Backpropogation with NumPy

## Initializing the weights

 - Weights should be small so that the input to the sigmoid is in the linear region near 0 and not at high and low ends. 
 - They also need to be initialized randomly so they all have different starting values and diverge. 
     - To do this, we can initialize the weights from a normal distribution centered at zero. A good value for the scale is $1/\sqrt{n}$ where $n$ is the number of input units. This keeps the input to the sigmoid low for increasing numbers of input units.

In code:

`weights = np.random.normal(scale=1/n_features**.5, size=n_features)`


## Data Prep

In [2]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('./data/admissions_data.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

## Gradient Descent

This is the general algorithm for updating the weights with gradient descent:

- Set the weight step to zero: $\Delta w_i = 0$
- For each record in the training data:
    - Make a forward pass through the network, calculating the output $\hat y = f(\sum_i w_i x_i)$
    - Calculate the error term for the output unit, $\delta = (y - \hat y) * f'(\sum_i w_i x_i)$
    - Update the weight step $\Delta w_i = \Delta w_i + \delta x_i$
    
- Update the weights $w_i = w_i + \alpha \Delta w_i / m$ where $\alpha$ is the learning rate and $m$ is the number of records. Below we will average the weight steps to help reduce any large variations in the training data.
- Repeat for $e$ epochs.

You can also update the weights on each record instead of averaging the weight steps after going through all the records.

In [3]:
# features is a dataframe, targets are series
# features is 360 x 6, targets is 360 x 1

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


def sigmoid_prime(x):
    """
    Calculate the derivative of the sigmoid, "sigmoid prime"
    """
    sigmoid_val = sigmoid(x)
    return sigmoid_val * (1 -sigmoid_val)

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
# Makes a nparray in the same size as the number of features (6: gre, gpa, rank_1, rank_2, rank_3, rank_4) 
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Calculate the prediction (before applying the sigmoid activation)
        h = np.dot(x, weights)
        
        # Calculate the output
        output = sigmoid(h)

        # Calculate the error
        error = y - output

        # Calculate the error term
        # sigmoid_prime = sigmoid_output * (1 -sigmoid_output)
        error_term = error * output * (1 - output)

        # Calculate the change in weights for this sample
        # and add it to the total weight change
        del_w += error_term * x

    # Update weights using the learning rate and the average change in weights
    weights += (learnrate * del_w) / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.2627609384996635
Train loss:  0.20928619409324875
Train loss:  0.20084292908073426
Train loss:  0.19862156475527873
Train loss:  0.1977985139668603
Train loss:  0.19742577912189863
Train loss:  0.1972350774624106
Train loss:  0.1971294562509248
Train loss:  0.19706766341315082
Train loss:  0.19703005801777368
Prediction accuracy: 0.725


# Backpropogation in NumPy

The general algorithm for backpropogation for a simple network with one hidden layer and one output layer.

- Set the weight steps for each layer to zero
    - The input to hidden weights $\Delta w_{ij} = 0$
    - The hidden to output weights $\Delta W_j = 0$
    
- For each record in the training data:
    - Make a forward pass through the network, calculating the output $\hat y$
    - Calculate the error gradient in the output unit, $\delta^o = (y - \hat y) f'(z)$ where $z = \sum_j W_j a_j$, the input to the output unit.
    - Propagate the errors to the hidden layer $\delta^h_j = \delta^o W_j f'(h_j)$
    - Update the weight steps:
        - $\Delta W_j = \Delta W_j + \delta^o a_j$
        - $\Delta w_{ij} = \Delta w_{ij} + \delta^h_j a_i$
        
- Update the weights, where $\alpha$ is the learning rate and $m$ is the number of records:
    - $W_j - W_j + \alpha\Delta W_j / m$
    - $w_{ij} - W_{ij} + \alpha\Delta w_{ij} / m$
    
- Repeat for $e$ epochs


Error term for the output layer:

$\delta_k = (y_k - \hat{y}_k)f^\prime(a_k)$

Error term for the hidden layer:

$\delta_ja = \sum [w_jk\delta_k]f^\prime(h_j)$

In [4]:
# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
print("Features shape: ", features.shape) # 360 x 6
last_loss = None

# Initialize weights
# 6 x 2
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
                                        
# 1 x 2
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)                    

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # x is a 1 x 6, one row in the data set
        
        # Done: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input) # 1 x 2
        output_layer_in = np.dot(hidden_output, weights_hidden_output)
        output = sigmoid(output_layer_in) # 1

        ## Backward pass ##
        # Done: Calculate the network's prediction error
        error = y - output

        # Done: Calculate error term for the output unit
        output_error_term = error * output * (1 - output)

        ## propagate errors to hidden layer

        # Calculate the hidden layer's contribution to the error
        hidden_error = np.dot(output_error_term, weights_hidden_output)
        
        # Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)
        
        # Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:, None]

    # Update weights  (don't forget to division by n_records or number of samples)
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Features shape:  (360, 6)
Train loss:  0.22586066779243663
Train loss:  0.22578766134845835
Train loss:  0.22571551799236633
Train loss:  0.22564422353474545
Train loss:  0.22557376408228183
Train loss:  0.22550412603125697
Train loss:  0.2254352960611944
Train loss:  0.22536726112865774
Train loss:  0.22530000846119533
Train loss:  0.2252335255514263
Prediction accuracy: 0.750
